In [ ]:
'''Convierte un archivo GIF a MP4 usando FFmpeg desde Python.  
'''
import subprocess

# Nombres de archivos
entrada = "video_prueba.gif"
salida = "animacion.mp4"

# Comando FFmpeg
comando = [
    "ffmpeg",
    "-y",  # sobrescribe si ya existe
    "-i", entrada,
    "-movflags", "faststart",
    "-pix_fmt", "yuv420p",
    "-vf", "scale=trunc(iw/2)*2:trunc(ih/2)*2",
    salida
]

# Ejecutar FFmpeg desde Python
subprocess.run(comando)

print(f"✅ Conversión completada: {salida}")


In [9]:
"""los fps son cuantos cuadros por segundo se muestran en un video"""
import cv2
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"fps: {fps}, frame count: {frame_count}, resolution: {width}x{height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('Video Playback', frame)
    if cv2.waitKey(int(1000/fps)) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@706.384] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


fps: 30.0, frame count: -1, resolution: 640x480


In [ ]:
'''este codigo hace lo contrario, guarda cada frame de un video en una carpeta'''
import cv2
import os

# Crear carpeta para los frames (si no existe)
os.makedirs('frames', exist_ok=True)

# Abrir el video
cap = cv2.VideoCapture('animacion.mp4')

frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f'frames/frame_{frame_id:04d}.jpg', frame)  
    frame_id += 1
    print(frame_id)   
cap.release()
cv2.destroyAllWindows()

print(f" {frame_id} frames guardados en la carpeta 'frames/'")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
 46 frames guardados en la carpeta 'frames/'


In [ ]:
'''este codigo une las imagenes guardadas en una carpeta y crea un video mp4'''
import cv2
import os
folder = 'frames'
frames = sorted([f for f in os.listdir(folder) if f.endswith(('.png', '.jpg'))])
first_frame = cv2.imread(os.path.join(folder, frames[0]))
height, width, _ = first_frame.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (width, height))
for frame_name in frames:
    frame = cv2.imread(os.path.join(folder, frame_name))
    out.write(frame)
out.release()

In [9]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('webcam read', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@2011.491] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [3]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('webcam read', gray_frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@7.692] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [8]:
'''Este codigo detecta un color especifico (azul verdoso) en la imagen de la webcam y muestra solo las areas con ese color'''
import cv2
import numpy as np  

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_teal = np.array([85, 80, 50])   
    upper_teal = np.array([100, 255, 255]) 

    mask = cv2.inRange(hsv_frame, lower_teal, upper_teal)
    teal_detected = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow('Deteccion Azul Verdoso', teal_detected)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@514.160] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [6]:
import cv2
import numpy as np

def draw_axis(img, center, vec, length, color, thickness=2):
    """
    Dibuja un vector (eigenvector) como una flecha desde 'center'
    con dirección 'vec' y escala 'length'.
    """
    # punto final escalado
    end = (int(center[0] + length * vec[0]), int(center[1] + length * vec[1]))
    cv2.arrowedLine(img, (int(center[0]), int(center[1])), end, color, thickness, tipLength=0.15)

cap = cv2.VideoCapture(0)

# FPS seguro (fallback si la cámara no reporta)
fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1:
    fps = 30
delay = int(1000 / fps)

print(f"fps: {fps}, frame count: N/A (cam), resolution: {int(cap.get(3))}x{int(cap.get(4))}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preproceso ligero
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Bordes y contornos
    edges = cv2.Canny(gray, 60, 180)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        # Tomamos el contorno más grande (feature más resaltante)
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)

        if area > 1500:  # umbral para evitar ruido
            # Resaltar el feature
            cv2.drawContours(frame, [c], -1, (0, 255, 255), 2)  # contorno en amarillo

            # Preparar puntos para PCA: Nx2 (x,y)
            data_pts = c.reshape(-1, 2).astype(np.float32)

            # PCA con OpenCV: media, eigenval/eigenvec
            mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean=None)

            # Centroide
            cx, cy = mean[0, 0], mean[0, 1]
            cv2.circle(frame, (int(cx), int(cy)), 4, (0, 0, 255), -1)  # centro rojo

            # Escalas para dibujar los ejes (proporcionales a la varianza)
            # sqrt de eigenvalues para que la longitud sea comparable a "desviación estándar"
            scale1 = float(np.sqrt(eigenvalues[0, 0])) * 2.0
            scale2 = float(np.sqrt(eigenvalues[1, 0])) * 2.0

            # Vectores propios (direcciones principales)
            v1 = eigenvectors[0]  # primer componente principal
            v2 = eigenvectors[1]  # segundo componente principal

            # Dibujar ejes PCA sobre la imagen original
            draw_axis(frame, (cx, cy), v1, scale1, (255, 0, 0), thickness=3)  # PC1 en azul (BGR)
            draw_axis(frame, (cx, cy), v2, scale2, (0, 255, 0), thickness=2)  # PC2 en verde

            # Etiquetas
            cv2.putText(frame, "Feature principal (contorno max)", (int(cx)+10, int(cy)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 3, cv2.LINE_AA)
            cv2.putText(frame, "Feature principal (contorno max)", (int(cx)+10, int(cy)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

            cv2.putText(frame, "PC1", (int(cx + scale1*v1[0]), int(cy + scale1*v1[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(frame, "PC2", (int(cx + scale2*v2[0]), int(cy + scale2*v2[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar
    cv2.imshow('PCA sobre feature mas resaltante', frame)
    #cv2.imshow('Bordes (Canny)', edges)

    if cv2.waitKey(delay) & 0xFF == 27:  # ESC para salir
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@400.609] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


fps: 30.0, frame count: N/A (cam), resolution: 640x480


In [9]:
import cv2
import numpy as np

def draw_axis(img, center, vec, length, color, thickness=2):
    end = (int(center[0] + length * vec[0]), int(center[1] + length * vec[1]))
    cv2.arrowedLine(img, (int(center[0]), int(center[1])), end, color, thickness, tipLength=0.2)

def pca_on_contour(cnt):
    pts = cnt.reshape(-1, 2).astype(np.float32)
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(pts, mean=None)

    cx, cy = mean[0, 0], mean[0, 1]
    lam1, lam2 = float(eigenvalues[0, 0]), float(eigenvalues[1, 0])
    var_ratio_pc1 = lam1 / (lam1 + lam2 + 1e-9)
    v1 = eigenvectors[0]  # dirección PC1
    v2 = eigenvectors[1]  # dirección PC2
    return (cx, cy), (v1, v2), (lam1, lam2), var_ratio_pc1

cap = cv2.VideoCapture(0)

fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1:
    fps = 30
delay = int(1000 / fps)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    disp = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(gray, 60, 180)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    contours = [c for c in contours if cv2.contourArea(c) > 1200]

    scored = []
    for c in contours:
        (cx, cy), (v1, v2), (lam1, lam2), vr1 = pca_on_contour(c)
        area = cv2.contourArea(c)
        score = area * lam1  
        scored.append({
            "cnt": c, "center": (cx, cy), "v1": v1, "v2": v2,
            "lam1": lam1, "lam2": lam2, "vr1": vr1, "score": score
        })

    scored.sort(key=lambda d: d["score"], reverse=True)

    for i, s in enumerate(scored):
        cx, cy = s["center"]
        v1, v2 = s["v1"], s["v2"]
        lam1, lam2, vr1 = s["lam1"], s["lam2"], s["vr1"]

        L1 = max(30, 2.0 * np.sqrt(lam1))
        L2 = max(20, 2.0 * np.sqrt(lam2))

        if i == 0:
            cv2.drawContours(disp, [s["cnt"]], -1, (255, 0, 255), 3)    
            cv2.circle(disp, (int(cx), int(cy)), 5, (255, 0, 255), -1)   
            draw_axis(disp, (cx, cy), v1, L1, (255, 0, 255), thickness=3)  
            draw_axis(disp, (cx, cy), v2, L2, (180, 0, 180), thickness=2)  
            cv2.putText(disp, f"TOP (PC1 var: {vr1*100:.1f}%)",
                        (int(cx)+10, int(cy)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (255, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.drawContours(disp, [s["cnt"]], -1, (0, 255, 255), 2)    
            cv2.circle(disp, (int(cx), int(cy)), 4, (0, 255, 255), -1)
            draw_axis(disp, (cx, cy), v1, L1, (255, 0, 0), thickness=2)  
            draw_axis(disp, (cx, cy), v2, L2, (0, 255, 0), thickness=1)  
            cv2.putText(disp, f"PC1 {vr1*100:.0f}%",
                        (int(cx)+6, int(cy)-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(disp, f"PC1 {vr1*100:.0f}%",
                        (int(cx)+6, int(cy)-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow("PCA: vectores + elemento mas caracteristico", disp)
    #cv2.imshow("Bordes (Canny)", edges)

    if cv2.waitKey(delay) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@702.582] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [13]:
import cv2
import numpy as np

def draw_axis(img, center, vec, length, color, thickness=2):
    """Dibuja una flecha representando un vector (eigenvector)"""
    end = (int(center[0] + length * vec[0]), int(center[1] + length * vec[1]))
    cv2.arrowedLine(img, (int(center[0]), int(center[1])), end, color, thickness, tipLength=0.2)

def pca_on_contour(cnt):
    """Calcula PCA sobre un contorno"""
    pts = cnt.reshape(-1, 2).astype(np.float32)
    mean, eigvecs, eigvals = cv2.PCACompute2(pts, mean=None)
    cx, cy = mean[0, 0], mean[0, 1]
    v1, v2 = eigvecs[0], eigvecs[1]
    lam1, lam2 = float(eigvals[0, 0]), float(eigvals[1, 0])
    return (cx, cy), (v1, v2), (lam1, lam2)

# === Inicialización de cámara ===
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1:
    fps = 30
delay = int(1000 / fps)

print("=== Detección de vectores PCA ===")
print("Presiona ESC para salir...\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    disp = frame.copy()

    # === Detección de bordes ===
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 60, 180)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)
        if area > 1500:
            (cx, cy), (v1, v2), (lam1, lam2) = pca_on_contour(c)

            # === Dibujar en la imagen original ===
            cv2.drawContours(disp, [c], -1, (0, 255, 255), 2)
            cv2.circle(disp, (int(cx), int(cy)), 5, (0, 0, 255), -1)

            L1 = max(30, 2.0 * np.sqrt(lam1))
            L2 = max(20, 2.0 * np.sqrt(lam2))

            draw_axis(disp, (cx, cy), v1, L1, (255, 0, 0), 3)  # PC1
            draw_axis(disp, (cx, cy), v2, L2, (0, 255, 0), 2)  # PC2

            # Etiquetas
            cv2.putText(disp, "PC1", (int(cx + L1*v1[0]), int(cy + L1*v1[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(disp, "PC2", (int(cx + L2*v2[0]), int(cy + L2*v2[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # === Mostrar valores en consola ===
            print(f"Centroide: ({cx:.2f}, {cy:.2f})")
            print(f"Vector PC1: [{v1[0]:.4f}, {v1[1]:.4f}]  |  λ1 = {lam1:.2f}")
            print(f"Vector PC2: [{v2[0]:.4f}, {v2[1]:.4f}]  |  λ2 = {lam2:.2f}")
            print(f"Razón λ1/λ2 = {lam1/lam2:.2f}")
            print("-" * 50)

    cv2.imshow("Original con vectores PCA", disp)

    if cv2.waitKey(delay) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@1064.821] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


=== Detección de vectores PCA ===
Presiona ESC para salir...

Centroide: (183.78, 198.97)
Vector PC1: [0.8577, 0.5142]  |  λ1 = 2308.29
Vector PC2: [-0.5142, 0.8577]  |  λ2 = 2165.79
Razón λ1/λ2 = 1.07
--------------------------------------------------
Centroide: (186.71, 199.49)
Vector PC1: [0.8870, 0.4618]  |  λ1 = 2313.21
Vector PC2: [-0.4618, 0.8870]  |  λ2 = 2196.62
Razón λ1/λ2 = 1.05
--------------------------------------------------
Centroide: (189.66, 198.60)
Vector PC1: [0.8975, 0.4410]  |  λ1 = 2324.01
Vector PC2: [-0.4410, 0.8975]  |  λ2 = 2221.88
Razón λ1/λ2 = 1.05
--------------------------------------------------
Centroide: (191.39, 196.70)
Vector PC1: [0.8769, 0.4807]  |  λ1 = 2326.94
Vector PC2: [-0.4807, 0.8769]  |  λ2 = 2253.41
Razón λ1/λ2 = 1.03
--------------------------------------------------
Centroide: (191.68, 193.15)
Vector PC1: [0.8879, 0.4601]  |  λ1 = 2335.36
Vector PC2: [-0.4601, 0.8879]  |  λ2 = 2245.62
Razón λ1/λ2 = 1.04
----------------------------------